# Complex Example

This notebook demonstrates advanced features of **flixopt** including:

- **Investment optimization** with piecewise cost functions
- **Status parameters** for startup costs and operating constraints
- **Piecewise conversion** for non-linear component behavior
- **Multiple effects** (costs, CO2, primary energy)
- **Saving and loading** results

This example builds a more realistic energy system model with operational constraints.

## Setup

In [1]:
import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## Configuration Options

These options allow experimenting with different model configurations:

In [2]:
# Experiment options
check_penalty = False  # Set True to test imbalance penalties
imbalance_penalty = 1e5  # High penalty for unmet demand
use_chp_with_piecewise_conversion = True  # Use piecewise CHP model

## Define Input Data

Create demand profiles and electricity prices:

In [3]:
# Demand profiles
electricity_demand = np.array([70, 80, 90, 90, 90, 90, 90, 90, 90])
heat_demand = (
    np.array([30, 0, 90, 110, 2000, 20, 20, 20, 20])
    if check_penalty
    else np.array([30, 0, 90, 110, 110, 20, 20, 20, 20])
)
electricity_price = np.array([40, 40, 40, 40, 40, 40, 40, 40, 40])

# Create time index
timesteps = pd.date_range('2020-01-01', periods=len(heat_demand), freq='h')

print(f'Heat demand range: {heat_demand.min()} - {heat_demand.max()} kW')

Heat demand range: 0 - 110 kW


## Create FlowSystem with Buses

We include an `imbalance_penalty` on each bus to handle cases where supply cannot meet demand:

In [4]:
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    fx.Bus('Strom', carrier='electricity', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Fernwärme', carrier='heat', imbalance_penalty_per_flow_hour=imbalance_penalty),
    fx.Bus('Gas', carrier='gas', imbalance_penalty_per_flow_hour=imbalance_penalty),
)

2025-12-08 14:27:17.969 INFO     │ Registered new Bus: Strom
2025-12-08 14:27:17.970 INFO     │ Registered new Bus: Fernwärme
2025-12-08 14:27:17.970 INFO     │ Registered new Bus: Gas


## Define Effects

Three effects track different aspects of the system:

In [5]:
# Costs - objective function with CO2 price linkage
Costs = fx.Effect(
    'costs',
    '€',
    'Kosten',
    is_standard=True,
    is_objective=True,
    share_from_temporal={'CO2': 0.2},  # 0.2 €/kg CO2
)

# CO2 emissions
CO2 = fx.Effect('CO2', 'kg', 'CO2_e-Emissionen')

# Primary energy with total limit
PE = fx.Effect('PE', 'kWh_PE', 'Primärenergie', maximum_total=3.5e3)

## Component 1: Gas Boiler with Investment

This boiler includes:
- **Investment parameters** with fixed size
- **Status parameters** for operating constraints (min/max uptime, startup costs)
- **Load constraints** (minimum and maximum part load)

In [6]:
Gaskessel = fx.linear_converters.Boiler(
    'Kessel',
    thermal_efficiency=0.5,
    status_parameters=fx.StatusParameters(
        effects_per_active_hour={Costs.label: 0, CO2.label: 1000}  # CO2 when running
    ),
    thermal_flow=fx.Flow(
        label='Q_th',
        bus='Fernwärme',
        size=fx.InvestParameters(
            effects_of_investment=1000,  # Fixed investment cost
            fixed_size=50,  # Fixed capacity
            mandatory=True,  # Must be built
            effects_of_investment_per_size={Costs.label: 10, PE.label: 2},  # Per-kW costs
        ),
        load_factor_max=1.0,
        load_factor_min=0.1,
        relative_minimum=5 / 50,
        relative_maximum=1,
        previous_flow_rate=50,  # Initial state
        flow_hours_max=1e6,  # Total energy limit
        status_parameters=fx.StatusParameters(
            active_hours_min=0,
            active_hours_max=1000,
            max_uptime=10,
            min_uptime=np.array([1, 1, 1, 1, 1, 2, 2, 2, 2]),  # Variable min uptime
            max_downtime=10,
            effects_per_startup={Costs.label: 0.01},  # Startup cost
            startup_limit=1000,
        ),
    ),
    fuel_flow=fx.Flow(label='Q_fu', bus='Gas', size=200),
)

## Component 2: Standard CHP

A simple CHP with constant efficiency ratios:

In [7]:
bhkw = fx.linear_converters.CHP(
    'BHKW2',
    thermal_efficiency=0.5,
    electrical_efficiency=0.4,
    status_parameters=fx.StatusParameters(effects_per_startup={Costs.label: 0.01}),
    electrical_flow=fx.Flow('P_el', bus='Strom', size=60, relative_minimum=5 / 60),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme', size=1e3),
    fuel_flow=fx.Flow('Q_fu', bus='Gas', size=1e3, previous_flow_rate=20),
)

## Component 3: Piecewise CHP

For more realistic modeling, efficiency can vary with load. The **piecewise conversion** defines segments where conversion ratios change:

```
P_el: [5-30] → [40-60]  (two operating segments)
Q_th: [6-35] → [45-100]
Q_fu: [12-70] → [90-200]
```

In [8]:
# Define flows for piecewise CHP
P_el = fx.Flow('P_el', bus='Strom', size=60, previous_flow_rate=20)
Q_th = fx.Flow('Q_th', bus='Fernwärme')
Q_fu = fx.Flow('Q_fu', bus='Gas')

# Define piecewise conversion with two operating segments
piecewise_conversion = fx.PiecewiseConversion(
    {
        P_el.label: fx.Piecewise([fx.Piece(5, 30), fx.Piece(40, 60)]),
        Q_th.label: fx.Piecewise([fx.Piece(6, 35), fx.Piece(45, 100)]),
        Q_fu.label: fx.Piecewise([fx.Piece(12, 70), fx.Piece(90, 200)]),
    }
)

bhkw_2 = fx.LinearConverter(
    'BHKW2',
    inputs=[Q_fu],
    outputs=[P_el, Q_th],
    piecewise_conversion=piecewise_conversion,
    status_parameters=fx.StatusParameters(effects_per_startup={Costs.label: 0.01}),
)

## Component 4: Storage with Piecewise Investment

Investment costs often have economies of scale. Here we model this with piecewise investment effects:

- Capacity range: 5-100 kWh
- Costs: non-linear (cheaper per kWh for larger storage)

In [9]:
# Piecewise investment effects (economies of scale)
segmented_investment_effects = fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise([fx.Piece(5, 25), fx.Piece(25, 100)]),
    piecewise_shares={
        Costs.label: fx.Piecewise([fx.Piece(50, 250), fx.Piece(250, 800)]),
        PE.label: fx.Piecewise([fx.Piece(5, 25), fx.Piece(25, 100)]),
    },
)

speicher = fx.Storage(
    'Speicher',
    charging=fx.Flow('Q_th_load', bus='Fernwärme', size=1e4),
    discharging=fx.Flow('Q_th_unload', bus='Fernwärme', size=1e4),
    capacity_in_flow_hours=fx.InvestParameters(
        piecewise_effects_of_investment=segmented_investment_effects,
        mandatory=True,
        minimum_size=0,
        maximum_size=1000,
    ),
    initial_charge_state=0,
    maximal_final_charge_state=10,
    eta_charge=0.9,
    eta_discharge=1,
    relative_loss_per_hour=0.08,
    prevent_simultaneous_charge_and_discharge=True,
)

## Define Sinks and Sources

In [10]:
# Heat demand
Waermelast = fx.Sink(
    'Wärmelast',
    inputs=[
        fx.Flow(
            'Q_th_Last',
            bus='Fernwärme',
            size=1,
            fixed_relative_profile=heat_demand,
        )
    ],
)

# Gas supply
Gasbezug = fx.Source(
    'Gastarif',
    outputs=[
        fx.Flow(
            'Q_Gas',
            bus='Gas',
            size=1000,
            effects_per_flow_hour={Costs.label: 0.04, CO2.label: 0.3},
        )
    ],
)

# Electricity feed-in
Stromverkauf = fx.Sink(
    'Einspeisung',
    inputs=[
        fx.Flow(
            'P_el',
            bus='Strom',
            effects_per_flow_hour=-1 * electricity_price,  # Revenue
        )
    ],
)

## Build and Visualize FlowSystem

In [11]:
# Add components
flow_system.add_elements(Costs, CO2, PE, Gaskessel, Waermelast, Gasbezug, Stromverkauf, speicher)

# Choose CHP type based on configuration
if use_chp_with_piecewise_conversion:
    flow_system.add_elements(bhkw_2)
else:
    flow_system.add_elements(bhkw)

# Print system overview
print(flow_system)

2025-12-08 14:27:18.037 INFO     │ Registered new Effect: costs
2025-12-08 14:27:18.037 INFO     │ Registered new Effect: costs
2025-12-08 14:27:18.037 INFO     │ Registered new Effect: CO2
2025-12-08 14:27:18.038 INFO     │ Registered new Effect: CO2
2025-12-08 14:27:18.038 INFO     │ Registered new Effect: PE
2025-12-08 14:27:18.038 INFO     │ Registered new Effect: PE
2025-12-08 14:27:18.039 INFO     │ Registered new Boiler: Kessel
2025-12-08 14:27:18.039 INFO     │ Registered new Sink: Wärmelast
2025-12-08 14:27:18.039 INFO     │ Registered new Source: Gastarif
2025-12-08 14:27:18.040 INFO     │ Registered new Sink: Einspeisung
2025-12-08 14:27:18.040 INFO     │ Registered new Storage: Speicher
2025-12-08 14:27:18.040 INFO     │ Registered new LinearConverter: BHKW2
FlowSystem
Timesteps: 9 (Hour) [2020-01-01 to 2020-01-01]
Periods: None
Scenarios: None
Status: ⚠

Components (6 items)
--------------------
 * BHKW2
 * Einspeisung
 * Gastarif
 * Kessel
 * Speicher
 * Wärmelast

Buses 

## Run Optimization

In [12]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01, time_limit_seconds=60))

2025-12-08 14:27:18.125 INFO     │ Auto-created Penalty Effect: Penalty
HighsMipSolverData::transformNewIntegerFeasibleSolution tmpSolver.run();
2025-12-08 14:27:20.127 INFO     │ Optimization solved successfully. Objective: -10711.5266


FlowSystem
Timesteps: 9 (Hour) [2020-01-01 to 2020-01-01]
Periods: None
Scenarios: None
Status: ✓

Components (6 items)
--------------------
 * BHKW2
 * Einspeisung
 * Gastarif
 * Kessel
 * Speicher
 * Wärmelast

Buses (3 items)
---------------
 * Fernwärme
 * Gas
 * Strom

Effects (4 items)
-----------------
 * CO2
 * costs
 * PE
 * Penalty

Flows (10 items)
----------------
 * BHKW2(P_el)
 * BHKW2(Q_fu)
 * BHKW2(Q_th)
 * Einspeisung(P_el)
 * Gastarif(Q_Gas)
 * Kessel(Q_fu)
 * Kessel(Q_th)
 * Speicher(Q_th_load)
 * Speicher(Q_th_unload)
 * Wärmelast(Q_th_Last)

## Save Results

Results can be saved to NetCDF format for later analysis:

In [13]:
# Save to file (uncomment to save)
# flow_system.to_netcdf('results/complex_example.nc')

## Analyze Results

In [14]:
flow_system.statistics.plot.heatmap('BHKW2(Q_th)')

PlotResult(data=<xarray.Dataset> Size: 152B
Dimensions:    (timeframe: 1, timestep: 9)
Coordinates:
  * timeframe  (timeframe) object 8B '2020-01-01'
  * timestep   (timestep) object 72B '00:00' '01:00' '02:00' ... '07:00' '08:00'
Data variables:
    value      (timestep, timeframe) float64 72B 45.0 45.0 64.6 ... 12.86 0.0, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>BHKW2(Q_th)|flow_rate: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2020-01-01'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00'], dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAACARkAAAAAAAIBGQK9ahEgoJl' ... 'AAAACARkAWKXZjubopQAAAAAAAAAAA'),
                    'dtype': 'f8',
                    'shape': '9, 1'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'BHKW2(Q_th)|flow_rate'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

In [15]:
flow_system.statistics.plot.balance('BHKW2')

PlotResult(data=<xarray.Dataset> Size: 288B
Dimensions:      (time: 9)
Coordinates:
  * time         (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-01T08:00:00
Data variables:
    BHKW2(Q_fu)  (time) float64 72B -90.0 -90.0 -129.2 ... -90.0 -25.73 -0.0
    BHKW2(P_el)  (time) float64 72B 40.0 40.0 47.13 60.0 ... 40.0 40.0 10.92 0.0
    BHKW2(Q_th)  (time) float64 72B 45.0 45.0 64.6 100.0 ... 45.0 45.0 12.86 0.0, figure=Figure({
    'data': [{'hovertemplate': 'variable=BHKW2(Q_fu)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'BHKW2(Q_fu)',
              'marker': {'color': '#1F77B4', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'BHKW2(Q_fu)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAACAVsAAAAAAAIBWwK9ahEgoJm' ... 'AAAACAVsAWKXZjubo5wAAAAAAAAACA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=BHKW2(P_el)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'BHKW2(P_el)',
              'marker': {'color': '#FECB52', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'BHKW2(P_el)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAREAAAAAAAABEQK4qA0wdkE' ... 'AAAAAAREAJahZE79UlQAAAAAAAAAAA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=BHKW2(Q_th)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'BHKW2(Q_th)',
              'marker': {'color': '#D62728', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'BHKW2(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAACARkAAAAAAAIBGQK9ahEgoJl' ... 'AAAACARkAWKXZjubopQAAAAAAAAAAA'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'bargap': 0,
               'bargroupgap': 0,
               'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'BHKW2 (flow_rate)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'time'}},
               'yaxis': {'anchor': 'x', 'dom

In [16]:
flow_system.statistics.plot.heatmap('Speicher')

PlotResult(data=<xarray.Dataset> Size: 152B
Dimensions:    (timeframe: 1, timestep: 9)
Coordinates:
  * timeframe  (timeframe) object 8B '2020-01-01'
  * timestep   (timestep) object 72B '00:00' '01:00' '02:00' ... '07:00' '08:00'
Data variables:
    value      (timestep, timeframe) float64 72B 0.0 13.5 52.92 ... 43.2 32.61, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>Speicher|charge_state: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2020-01-01'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00'], dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAAAAABAAAAAAArQPYoXI/CdU' ... 'AAAACANkCamZmZmZlFQPSmN73pTUBA'),
                    'dtype': 'f8',
                    'shape': '9, 1'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Speicher|charge_state'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

In [17]:
flow_system.statistics.plot.balance('Fernwärme')

PlotResult(data=<xarray.Dataset> Size: 432B
Dimensions:                (time: 9)
Coordinates:
  * time                   (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-0...
Data variables:
    Kessel(Q_th)           (time) float64 72B -2.914e-15 ... -2.914e-15
    Speicher(Q_th_unload)  (time) float64 72B -0.0 7.105e-15 ... -7.135 -20.0
    BHKW2(Q_th)            (time) float64 72B -45.0 -45.0 -64.6 ... -12.86 -0.0
    Wärmelast(Q_th_Last)   (time) float64 72B 30.0 0.0 90.0 ... 20.0 20.0 20.0
    Speicher(Q_th_load)    (time) float64 72B 15.0 45.0 ... 9.869e-16 -7.895e-15, figure=Figure({
    'data': [{'hovertemplate': 'variable=Kessel(Q_th)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Kessel(Q_th)',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Kessel(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAABA6rwAAAAAAEDqvAAAAAAAQO' ... 'AAAABA6rwAAAAAAEDqvAAAAAAAQOq8'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Speicher(Q_th_unload)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Speicher(Q_th_unload)',
              'marker': {'color': '#FFA15A', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Speicher(Q_th_unload)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAIAAAAAAAAAAPUSV7t1eZz' ... 'AAAAAAAIDJrRM5jYocwP3//////zPA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=BHKW2(Q_th)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'BHKW2(Q_th)',
              'marker': {'color': '#19D3F3', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'BHKW2(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAACARsAAAAAAAIBGwK9ahEgoJl' ... 'AAAACARsAWKXZjubopwAAAAAAAAACA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Wärmelast(Q_th_Last)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Wärmelast(Q_th_Last)',
              'marker': {'color': '#EF553B', 

## Summary

This example demonstrated advanced flixopt features:

- **InvestParameters** for component sizing optimization
- **StatusParameters** for operational constraints (min uptime, startup costs)
- **PiecewiseConversion** for non-linear component behavior
- **PiecewiseEffects** for economies of scale in investment
- **Multiple effects** with constraints and cross-linkage
- **Result persistence** via NetCDF

For even more advanced optimization modes, see the optimization modes notebook!